In [76]:
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
from os import listdir
from os.path import isfile, join
from decimal import *

In [241]:
class NaiveBayes():
    def __init__(self):
        self.words = {}
        self.words["legit"] = {"subj" : {}, "body" : {}}
        self.words["spam"] = {"subj" : {}, "body" : {}}
        self.total_count = 0
        self.count = {"legit" : 0, "spam" : 0}
        
    def __calc_prob(self, cls):
        #total = sum(list(words[cls]["subj"].values())))
        total = Decimal(sum(list(self.words[cls]["body"].values())))
        self.words[cls]["prob"] = {}
        for word, count in self.words[cls]["body"].items():
            self.words[cls]["prob"][word] = Decimal(count) / total
    
        
    def train(self, mails):
        self.total_count = len(mails)
        for mail in mails:
            cls, subj, body = mail
            self.count[cls] += 1
            for w in subj:
                self.words[cls]["subj"][w] = self.words[cls]["subj"].get(w, 0) + 1
            for w in body:
                self.words[cls]["body"][w] = self.words[cls]["body"].get(w, 0) + 1
        self.__calc_prob("legit")
        self.__calc_prob("spam")
        
    def classify(self, subj, body):
        score_legit = Decimal(self.count["legit"]) / Decimal(self.total_count)
        score_spam = Decimal(self.count["spam"]) / Decimal(self.total_count)
        #pprint((score_legit, score_spam))
        for word in body:
            if word in self.words["legit"]["body"] and word in self.words["spam"]["body"]:
                score_legit *= self.words["legit"]["prob"].get(word, Decimal(1))
                score_spam *= self.words["spam"]["prob"].get(word, Decimal(1))
        #pprint ((score_legit, score_spam))
        if score_spam > score_legit:
            return "spam"
        return "legit"

In [228]:
def read_mail(filename):
    lines = [line[:-1] for line in open(filename, "r").readlines()]
    subj = lines[0][len("Subject: "):].split(" ")
    if subj and subj[0]:
        subj = [int(i) for i in subj]
    else:
        subj = []
    body = lines[2].split(" ")
    if body and body[0]:
        body = [int(i) for i in body]
    else:
        body = []
    cls = "legit" if "legit" in filename else "spam"
    return (cls, subj, body)

def files_in_block(index):
    mypath = "data/part{}/".format(index)
    onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return onlyfiles

In [229]:
def f1score(theory, practice):
    tp, fn, fp, tn = 0, 0, 0, 0
    for i in range(len(theory)):
        tp += theory[i] == practice[i] and practice[i] == 1
        tn += theory[i] == practice[i] and practice[i] == 0
        fn = sum(practice) - tp
        fp = (len(practice) - sum(practice)) - tn
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return (2 * precision * recall / (precision + recall), fn, fp)

In [242]:
all_files = []
for i in range(1, 10):
    for f in files_in_block(i):
        all_files.append(f)
mails = [read_mail(m) for m in all_files]
classifier = NaiveBayes()
classifier.train(mails)

In [243]:
mail = [read_mail(f) for f in files_in_block(10)] 
true_y = list(map(lambda x: 1 if x is "spam" else 0, [i[0] for i in mail]))
data = [(i[1], i[2]) for i in mail]

In [244]:
predicted = [classifier.classify(i[0], i[1]) for i in data]
predicted_y = list(map(lambda x: 1 if x is "spam" else 0 , predicted))

In [245]:
f1score(ans, predicted_y)

(0.9306930693069307, 6, 1)

In [250]:
for i in range(len(true_y)):
    if true_y[i] != predicted_y[i] and predicted_y[i] == 0:
        print(i)

55


In [249]:
mail[5]

('legit',
 [3735, 24183, 6567, 14108, 10930, 773, 82, 9118],
 [284,
  10092,
  9561,
  17798,
  80,
  2176,
  5657,
  9472,
  2130,
  19163,
  17345,
  18957,
  16502,
  6567,
  14338,
  16502,
  12359,
  3735,
  24183,
  13472,
  14108,
  1835,
  97,
  69,
  84,
  23607,
  9583,
  15964,
  9408,
  13665,
  1663,
  18776,
  13383,
  1967,
  18996,
  2130,
  2170,
  1991,
  19206,
  84,
  1812,
  17345,
  4339,
  6427,
  17753,
  16502,
  6567,
  80,
  23607,
  4520,
  19054,
  7634,
  84,
  23266,
  24097,
  80,
  15031,
  13406,
  2377])